<a href="https://colab.research.google.com/github/DorShabat/Cryptology-Project/blob/main/Schnorr_signature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Schnorr signature

## pips:

In [2]:
%%capture
!pip install cryptography
!pip install gmpy2

##imports:

In [3]:
import numpy as np
import sympy
import os
import random
import hashlib
from sympy.ntheory.generate import randprime
import gmpy2

## functions for generating the mathematical parameters:

In [4]:
def get_factor_of_prime_minus_one(prime): # get the factor of prime - 1
    if not sympy.isprime(prime):
        raise ValueError("The input number is not a prime number.")
    p_minus_1 = prime - 1
    factors = sympy.factorint(p_minus_1)
    return max(factors.keys())


def mod_exp(base, exp, mod): # modular exponentiation
    base = gmpy2.mpz(base)
    exp = gmpy2.mpz(exp)
    mod = gmpy2.mpz(mod)
    return gmpy2.powmod(base, exp, mod)


def find_A(Q, P): # find the non-trivial solution to A^Q = 1 mod P
    if Q == 0:
        raise ValueError("Q must be non-zero.")
    if P <= 1:
        raise ValueError("P must be greater than 1.")
    A = gmpy2.mpz(2)
    while A < P:
        if mod_exp(A, Q, P) == 1:
            return A
        A += 1
    raise ValueError("No non-trivial solution found.")


def mod_inverse(A, Q): # find the modular inverse of A under modulo Q
    A = gmpy2.mpz(A)
    Q = gmpy2.mpz(Q)
    g, x, y = extended_gcd(A, Q)
    if g != 1:
        raise ValueError(f"Modular inverse does not exist for A={A} and Q={Q}")
    else:
        return x % Q


def extended_gcd(a, b): # Extended Euclidean Algorithm
    if a == 0:
        return b, 0, 1
    else:
        g, x1, y1 = extended_gcd(b % a, a)
        x = y1 - (b // a) * x1
        y = x1
        return g, x, y


def calculate_V(A, s, P): # Calculate V as A^(-s) mod P
    A = gmpy2.mpz(A)
    s = gmpy2.mpz(s)
    P = gmpy2.mpz(P)
    V = gmpy2.powmod(A, -s, P)
    return V


def hash_function(data): # Hash function
    if isinstance(data, str):
        data = data.encode('utf-8')
    sha256 = hashlib.sha256()
    sha256.update(data)
    return sha256.hexdigest()

##Networking:

# Alice

***Sign:***

`M = message to be signed`

global:
```
P = a prime number     # typically 1024-bit
Q = a factor of P-1    # typically 160-bit
A = a^Q === 1 mod P
```
private key:
`s = random 0 < s < Q`

public verification key:
`V = A^(-s) mod P`

for signing:
```
r = random 0 < r < Q
x = A^r mod P
e = Hash(M||x)
y = (r+se)modQ
```

Send: ` Message, M | Signature(e , y) `


### generate parameters:

In [5]:
M = 'This is a test message with my credit card digits.' # binary msg

# P = randprime(2**1023, 2**1024 - 1)
P = randprime(2**34, 2**35 - 1)                # typicalyy 1024-bit number
Q = get_factor_of_prime_minus_one(P)           # typically 160-bit number
A = find_A(Q, P)
s = random.randint(1, Q-1)
V = calculate_V(A, s, P)
r = random.randint(1, Q-1)
x = mod_exp(A, r, P)
e = hash_function(str(M) + str(x))
y = (r + s*int(e, 16)) % Q

### parameters printing:

In [6]:
print(f'P = {P}')
print(f'Q = {Q}')
print(f'A = {A}')
print(f's = {s}')
print(f'V = {V}')
print(f'r = {r}')
print(f'x = {x}')
print(f'e = {e}')
print(f'y = {y}')

P = 20188783591
Q = 672959453
A = 10
s = 212802307
V = 11756512537
r = 84734927
x = 16429335271
e = 7fe8af7fe390f626af452762766e8ccab45e4c70e07356d3a293a0ef2a78ce6a
y = 573828376


# Bob


*** Verification:***
  
Received:
`(M,  e,  y)`

Known publicly:
`(A,  P,  Q,  V)`

compute:
* `x' = (A^y * V^e) mod P`
* `e' = H( M || x' )`

### print what recived and known:

In [7]:
print("   Recived form alice:\n")
print(f'M = {M}')
print(f'e = {e}')
print(f'y = {y}')
print("\n\n  known public parameters:\n")
print(f'A = {A}')
print(f'P = {P}')
print(f'Q = {Q}')
print(f'V = {V}')

   Recived form alice:

M = This is a test message with my credit card digits.
e = 7fe8af7fe390f626af452762766e8ccab45e4c70e07356d3a293a0ef2a78ce6a
y = 573828376


  known public parameters:

A = 10
P = 20188783591
Q = 672959453
V = 11756512537


### compute:

In [8]:
Ay = mod_exp(A, y, P)
Ve = mod_exp(V, int(e, 16), P)
AyVe = (Ay * Ve) % P
x_tag = AyVe

print(f"Alice`s x value is {x} and Bob`s x' value is {x_tag}")
if(x == x_tag):
  print("so, (x = x')")

Alice`s x value is 16429335271 and Bob`s x' value is 16429335271
so, (x = x')


Technically, Bob should check if his computed x' is equal to Alice's x.

`(x' == x)` But he doen't know Alice`s x value beacuse it private.

So, Bob will Compute the e` by the hash function value and compare it with Alice's e value.

`e' = H( M || x' )`

In [9]:
e_tag = hash_function(str(M) + str(x_tag))

print(f"Alice`s e = {e}")
print(f" Bob`s e' = {e_tag}\n")

if(e == e_tag):
  print("so, (e = e')")
  print("and it means the message M was really signed by Alice.")
else:
  print("so, (e != e')")
  print("and it means the message M was not signed by Alice or it was alerted by someone in the way.")


Alice`s e = 7fe8af7fe390f626af452762766e8ccab45e4c70e07356d3a293a0ef2a78ce6a
 Bob`s e' = 7fe8af7fe390f626af452762766e8ccab45e4c70e07356d3a293a0ef2a78ce6a

so, (e = e')
and it means the message M was really signed by Alice.
